<div class="alert alert-block alert-info">
<i>Hello User!<br></i>
Welcome to <b>python-iLOrest-library</b> <br> 
This script can be used to retrieve the <b>base registry information</b> from an HPE iLO (Integrated Lights Out) server.  <br> 
To perform this you need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "background-color:white">python-ilorest-library</a> installed in your machine.<br>
And please keep your username and password handy!
<br>
<b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class = "alert alert-block alert-success">
    <div style = "color:green; font-size:40px;">1.</div>
    <br>
    First, we need to "import" certain files that provide us the functionality to get the base registry.</div>

In [ ]:
import sys
from redfish import LegacyRestClient
from get_resource_directory import get_resource_directory


<div class = "alert alert-block alert-success">
    <div style = "color:green; font-size:40px;">2.</div><br>
    Below is the get_base_registry function. The get_base_registry function serves the purpose of retrieving the base registry information from an HPE iLO server using the HPE iLO REST API.<br> 
    It performs the following steps to accomplish this:<br><br>
1. The get_base_registry function sends a GET request to the /rest/v1/Registries endpoint of the HPE iLO server to retrieve the registry data.
<br><br>
2. It iterates through the response entries, checking for the "Id" field or extracting the schema name.
<br><br>
3. For each entry, it makes additional GET requests to the specified URIs to retrieve the registry information.
<br><br>
4. If successful, it displays a message and stores the registry messages in a dictionary. The function returns the dictionary containing the registry messages.
<br><br>
In summary, the get_base_registry function retrieves the base registry information by making requests to specific URIs, extracting relevant data, and storing it for further analysis and management.
<br>
</div>

In [ ]:
def get_base_registry(restobj):
    response = restobj.get("/rest/v1/Registries")
    messages = {}

    identifier = None

    for entry in response.dict["Items"]:
        if "Id" in entry:
            identifier = entry["Id"]
        else:
            identifier = entry["Schema"].split(".")[0]

        if identifier not in ["Base", "iLO"]:
            continue

        for location in entry["Location"]:
            reg_resp = restobj.get(location["Uri"]["extref"])

            if reg_resp.status == 200:
                sys.stdout.write("\tFound " + identifier + " at " + \
                                            location["Uri"]["extref"] + "\n")
                messages[identifier] = reg_resp.dict["Messages"]
            else:
                sys.stdout.write("\t" + identifier + " not found at "\
                                            + location["Uri"]["extref"] + "\n")

    return messages

<div class = "alert alert-block alert-success">
    <div style = "color:green; font-size:40px;">3.</div><br>
The following code establishes a connection to an HPE iLO server using the HPE iLO REST API. It sets up the necessary parameters for either local or remote server connection, creates a REST client object, logs into the iLO server, executes an example function to retrieve registry information, and logs out from the server. This code showcases how to interact with the iLO server programmatically, demonstrating the usage of the HPE iLO REST API for server management tasks.
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    #Create a REST object
    REST_OBJ = LegacyRestClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD)
    REST_OBJ.login()
    sys.stdout.write("\nEXAMPLE 2: Find and return registry " + "\n")
    get_base_registry(REST_OBJ)
    REST_OBJ.logout()

<div class = "alert alert-block alert-warning">
    <b>When running on the server locally</b> use the following values<br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>